# Scikit-decide performance

Scheduling solvers run an external program, minizinc, to find the solution.  Solve may take a long time, but there is little we can do inside scikit-decide, users will have either to change some parameters or to select another solver.
But after a solution is found, rollout may not be as fast as it should.  This notebook explores some ways to make rollouts run faster.

## Methodology

A `computation_time_experiments.py` script was provided, to compare timings of rollouts when `Domain.set_inplace_environment` method has been called with either `True` or `False` argument.

On my machine, the following timings are found with `j1201_1.sm`:
* inplace=False: 0.5128
* inplace=True: 0.0298

A 2nd comparison is also performed, when all durations are multiplied by 20.  Timings become
* inplace=True: 1.4344
* inplace=False: 0.1443

In order to help with profiling, the first action is to modify the script so that we can pass the `inplace` argument on command-line.
* `--graph_file <FILE>`:  filename to load
* `--inplace 0|1`: runs rollout with inplace set to False or True (or both if flag is unused)
* `--makespan <N>`: multiply all durations by N
* `--plot`: plot Gantt diagrams, may be useful for validation
* `--output <FILE>`: store in a text file the `t` member of each state of the solution, may also be useful for validation purposes

A quick analysis with profiling show that almost all the overhead is caused by `State.copy()`, so we will focus on it.
Hand-made timers are added into this method to display time of copy by State member.  Since computations are quite fast, we also use the largest graph file found in this repository,
`j12052_1.sm`.  It is very important to understand that we do not rely on these timers to assess improvements, they must be measured on code without timers.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 1
0.0420 seconds to rollout policy with inplace=True environment, Final time of the schedule : 205    
Timers deepcopy Times: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] total = 0
Count: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 1 --makespan 1
0.6511 seconds to rollout policy with inplace=False environment, Final time of the schedule : 205
Timers deepcopy Times: [0.009053707122802734, 0.0019555091857910156, 0.0012674331665039062, 0.01271820068359375, 0.01219630241394043, 0.0072896480560302734, 0.0005247592926025391, 0.0012519359588623047, 0.0011410713195800781, 0.0035338401794433594, 0.0012655258178710938, 0.5308926105499268, 0, 0, 0] total = 0.5830905437469482
Count: [19259, 858, 0, 20635, 20635, 19777, 0, 1308, 1308, 858, 0, 39894, 0, 0, 0]
```

For each run, we print:
1. the total time spent in rollout
1. the time spent to copy each member, and the sum of these numbers
1. the cumulated size of each member

For reference, the list of members is
```
tasks_remaining
tasks_ongoing
tasks_paused
tasks_progress
tasks_mode
tasks_complete
resource_to_task
resource_availability
resource_used
resource_used_for_task
_current_conditions
tasks_details
```
followed by 3 zeros (arrays for timers had been overallocated in order to allow adding more members).

We can see that few arrays are empty but take some time to copy.  This may be an artifact of our timers, and since these times are several orders of magnitude lower than the largest ones, they are ignored.  This is also why it is important to benchmark our results without these timers. 

This is consistent with previous analysis, most of the time is spent in copy (0.583s on 0.651s total time).  There is also some overhead in other methods since 0.651-0.583=0.068 > 0.042, but it is worthwhile to first focus on copy.  We can try other values of `--makespan` to see if we can find a bigger difference, and indeed for 3 we have

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 1 --makespan 3
0.0977 seconds to rollout policy with inplace=True environment, Final time of the schedule : 615
Timers deepcopy Times: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] total = 0
Count: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 3
1.4763 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.019394636154174805, 0.00424957275390625, 0.0027191638946533203, 0.028195619583129883, 0.027329444885253906, 0.016315460205078125, 0.0011255741119384766, 0.002678394317626953, 0.002628803253173828, 0.00837087631225586, 0.002740621566772461, 1.2144522666931152, 0, 0, 0] total = 1.330200433731079
Count: [42771, 2096, 0, 47143, 47143, 45047, 0, 2948, 2948, 2096, 0, 89914, 0, 0, 0]
```

Difference is not higher with very large spans, but overall computations take more time, this issue will be discussed at the end of this notebook.

## Reduce copy time

A first observation is that copying `tasks_details` takes 90% of rollout time.  This member is a `Dict[int, Task]`.

### Remove resources from tasks_details

This member seems to be unused.  This change is trivial, it is performed first.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 1 --makespan 3
1.3194 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.021335124969482422, 0.004431247711181641, 0.0029480457305908203, 0.03144526481628418, 0.030480384826660156, 0.018048763275146484, 0.00119781494140625, 0.00293731689453125, 0.0028183460235595703, 0.009209156036376953, 0.003084897994995117, 1.0345032215118408, 0, 0, 0] total = 1.1624395847320557
Count: [42771, 2096, 0, 47143, 47143, 45047, 0, 2948, 2948, 2096, 0, 89914, 0, 0, 0]
```

Looks nice.  Can we drop this member?

### Lazy initialization of tasks_details

This one is much more tricky.  In State constructor, `tasks_details`  member is initialized for each available task.  There are 122 tasks, and 89914=737*122, which means that we created 737 State instances.  Dict values are a complex object (Task), and thus initialization takes some time.  In this commit we put Task inside `tasks_details` only when task is started.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 3
0.6967 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.019779443740844727, 0.003989458084106445, 0.0026955604553222656, 0.027713537216186523, 0.026833772659301758, 0.016098976135253906, 0.0011124610900878906, 0.0026047229766845703, 0.0024499893188476562, 0.008087635040283203, 0.0027413368225097656, 0.4784829616546631, 0, 0, 0] total = 0.5925898551940918
Count: [42771, 2096, 0, 47143, 47143, 45047, 0, 2948, 2948, 2096, 0, 47143, 0, 0, 0]
```

With this change, tasks_details copy goes from 1.03s down to 0.48s.

### Even smarter tasks_details

The next idea is to realize than when tasks are completed, the same informations in tasks_details are copied over and over.  A new member `tasks_complete_details` is added, and informations are moved from `tasks_details`into `tasks_complete_details` when task is completed.  For now this won't change anything, but it will help us to understand if we have to focus on `tasks_details` or `tasks_complete_details`.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 3
0.7191 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.019849061965942383, 0.004220485687255859, 0.002743244171142578, 0.028657197952270508, 0.0274965763092041, 0.016595840454101562, 0.0011150836944580078, 0.0027115345001220703, 0.0026586055755615234, 0.008584737777709961, 0.0026755332946777344, 0.02179741859436035, 0.469163179397583, 0, 0] total = 0.6082684993743896
Count: [42771, 2096, 0, 47143, 47143, 45047, 0, 2948, 2948, 2096, 0, 2096, 45047, 0, 0]
```

First of all, a warning to state that total time (0.719s) is larger than the previous one (0.697s), but differences are not significant, if we compare on many runs we will find that they are similar.  Now the intersting part is that copy of `tasks_complete_details` takes much more time than `tasks_details`.  So we know that informations stored in `tasks_complete_details` won't change during processing, all we want to do is to be able to add new items without copying the previous ones.  A solution is to use a singly linked list.  When a new item is added, it is inserted at the head and points to the previous head.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 3
0.2683 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.020892620086669922, 0.003971576690673828, 0.0025665760040283203, 0.028470516204833984, 0.027367591857910156, 0.017148733139038086, 0.0010986328125, 0.0026979446411132812, 0.00252532958984375, 0.007909774780273438, 0.02013254165649414, 0.023048877716064453, 0.005220651626586914, 0, 0] total = 0.16305136680603027
Count: [42771, 2096, 0, 47143, 47143, 45047, 0, 2948, 2948, 2096, 0, 2096, 45047, 0, 0]
```

That change looks pretty cool, time to copy `tasks_details` and `tasks_complete_details` is down from 0.48s (before lazy initialization) down to 0.028s.  But be aware that

1. this change breaks many things, because other classes expect `tasks_details` to contain everything from `tasks_complete_details`; a hack has been applied to `computation_time_experiments.py` to work around this problem, but more changes are required
1. lookup and size becomes O(N) operations on `tasks_complete_details`.  This may cause severe slowdown, we have to ensure that these operations are not performed

If we take a look at the current list of members
```
tasks_remaining
tasks_ongoing
tasks_paused
tasks_progress
tasks_mode
tasks_complete
resource_to_task
resource_availability
resource_used
resource_used_for_task
_current_conditions
tasks_details
tasks_complete_details
```

we see that next candidates are `tasks_progress`, `tasks_mode` and `tasks_remaining`.

### More singly linked lists

The first two can be replaced by singly linked lists.  There is not much to say, except that these changes have been performed when writing this document and had not been well tested, these changes.  Anyway results look interesting.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 3
0.2307 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.020183563232421875, 0.004435539245605469, 0.002676725387573242, 0.002731800079345703, 0.002198934555053711, 0.017505407333374023, 0.0011265277862548828, 0.002683877944946289, 0.0025451183319091797, 0.008571147918701172, 0.002852916717529297, 0.040422916412353516, 0.005220651626586914, 0.0042650699615478516, 0.003795146942138672] total = 0.1212153434753418
Count: [42771, 2096, 0, 2096, 2096, 45047, 0, 2948, 2948, 2096, 0, 2096, 45047, 45047, 45047]
```

I do not understand why time for `tasks_complete_details` has now doubled.

### Remove tasks_remaining

For obvious reasons the same split cannot be perforled on `tasks_remaining`.
The idea behind `tasks_remaining` is that some tasks may require conditions which are not fulfilled, so `tasks_remaining` is a subset of `task_ids` which contains tasks which can really be started. At the beginning of simulation, this set may be large, and is copied into all State instances.  We replace this member by a new member `tasks_unsatisfiable`, a set containing the ids of tasks for which canditions are not fulfilled. We can use `tasks_unsatisfiable` to compute `tasks_remaining`:

```
tasks_remaining = task_ids - tasks_complete - tasks_progress - tasks_unsatisfiable
```

Copy of `tasks_unsatisfiable` will be much faster than `tasks_remaining` because this set will be slower, but the extra computations may cancel this gain, the overall speedup (if any) must be confirmed on real cases.

```nocolor
$ python computation_time_experiments.py --graph_file ../../notebooks/j12052_1.sm  --inplace 0 --makespan 3
0.2250 seconds to rollout policy with inplace=False environment, Final time of the schedule : 615
Timers deepcopy Times: [0.006215572357177734, 0.004292726516723633, 0.002567768096923828, 0.0027618408203125, 0.0022406578063964844, 0.017495393753051758, 0.0010764598846435547, 0.002802610397338867, 0.0025997161865234375, 0.008543968200683594, 0.002688169479370117, 0.039988040924072266, 0.0050771236419677734, 0.004088878631591797, 0.003808259963989258] total = 0.1062471866607666
Count: [0, 2096, 0, 2096, 2096, 45047, 0, 2948, 2948, 2096, 0, 2096, 45047, 45047, 45047]
```

This does not seem to change anything.  Maybe this requires more work to see if operations can be rewritten to not require those extra computations.

It is time to summarize our findings so far.  We remove timers so that results are not polluted, and we compare timings for span 1, 3 and 20:

| | old inplace=F  | new inplace=F | old inplace=T | new inplace=T |
|---|:-------------:|:-----------:|:------:|:------:|
| span 1 | 0.6322 | 0.0990       | 0.0400   | 0.0471     |
| span 3 | 1.4395 | 0.2064       | 0.0946   | 0.1124     |
| span 20 | 1.2969 | 0.2123       | 0.1274   | 0.1378     |

New code is a little bit slower when inplace is True, probably because of the addition of extra members, but is much faster when inplace is False, which is the most common use case.

## Other improvements

If we take another look at profiling, we see that `MixedRenewable.all_tasks_possible()` takes some time.  It contains the following code:

```python
for task_id in remaining_tasks:
    task_possible = True
    for mode in self.get_task_modes(task_id).keys():
        mode_possible = True
        for res in self.get_resource_types_names():
            if not self.get_resource_renewability()[res]:
                need = self.get_task_modes(task_id)[mode].get_resource_need(res)
                avail = (
                    state.resource_availability[res] - state.resource_used[res]
                )
                if avail - need < 0:
                    mode_possible = False 
                    break
        if mode_possible is True:
            break
    if not mode_possible:
        return False
return True
```

The problem here is that `self.get_task_modes(task_id)` in the innermost loop is equivalent to `self.get_tasks_modes()[task_id]`; so we create a large dictionary just to retrieve a single value.  If we store the result of `self.get_tasks_mode()` before entering the loops and reuse that map, this code runs much faster:

```python
resource_not_renewable = set(
    res
    for res, renewable in self.get_resource_renewability().items()
    if res in resource_types_names and not renewable
)
avail = {
    res: state.resource_availability[res] - state.resource_used[res]
    for res in resource_not_renewable
}
modes_details = self.get_tasks_modes()
for task_id in remaining_tasks:
    for mode_consumption in modes_details[task_id].values():
        if all(
            avail[res] >= mode_consumption.get_resource_need(res)
            for res in resource_not_renewable
        ):
            break
    else:
        # The else-clause runs if loop completes normally, which means
        # that we did not find a mode for which all resources are available.
        return False
return True
```

There are several key points in this code:
 * this error would not have been made if `get_task_mode` did not exist; such methods encourage users to make this mistake.  This is also true for `MixedRenewable.is_renewable()` which calls `get_resource_renewability()`.  And probably many other methods should be removed too.
 * constant objects should always be defined outside of loops
 * there is no need to call keys() when iterating on a dict
 * when iterating on a dict, if `dict[key]` is read inside this loop, it is always better to replace it by `for key, val in map.items()`
 * the `else` clause of `for`-loops is not well known but pretty handy

With these changes, code with inplace=False is almost as fast as old code with inplace=True:

| | new inplace=F | new inplace=T | old inplace=T |
|---|:------:|:------:|:-----:|
| span 1 | 0.0668   | 0.0287     | 0.0400  |
| span 3 | 0.1646   | 0.0648     | 0.0946  |
| span 20 |  0.1467   | 0.0712     | 0.1274  |

## Digressions

It is a common error to believe that Numpy is always faster than plain Python.  Numpy operations are faster on Numpy arrays, but not on Python lists because they have to be converted into a Numpy array.

In [ ]:
import numpy as np

In [ ]:
%%timeit
np.sum([i for i in range(100)])

In [ ]:
%%timeit
sum([i for i in range(100)])

There are several occurrences of this pattern, for instance in `rcpsp_model.py`.  This file also contains

    resource_avail_in_time[res] = np.full(new_horizon, self.problem.resources[res], dtype=int).tolist()

which could be written as

    resource_avail_in_time[res] = [int(self.problem.resources[res])] * new_horizon

But if Numpy arrays are defined before entering the loop, computations become much faster.  The whole script ran on 100s with a span of 100000, and on 2s with Numpy arrays.
Code in `generate_schedule_from_permutation_serial_sgs` and `generate_schedule_from_permutation_serial_sgs_2` could probably be improved too.

More generally, I do not feel comfortable with all these dicts; tasks, modes and resources have all a fixed size, so it is likely that most of these dicts could be replaced by Numpy arrays.